## Imports

https://github.com/IBMDecisionOptimization/docplex-examples/blob/master/examples/mp/jupyter/tutorials/Linear_Programming.ipynb


Integer Optimization:
- https://github.com/IBMDecisionOptimization/docplex-examples/blob/master/examples/mp/jupyter/tutorials/Beyond_Linear_Programming.ipynb

In [45]:
from docplex.mp.model import Model
import numpy as np
import pandas as pd

In [46]:
# load data

players = pd.read_csv("../data/processed-data/0_selection_players.csv", index_col=0)
n_players = players.shape[0]
salaries = pd.read_csv("../data/processed-data/0_selection_salary.csv", index_col=0,).drop('Player', axis=1).values
n_time = salaries.shape[1]
n_trials = int(salaries.shape[0]/n_players)
salaries = salaries.reshape((n_trials, n_players, n_time))
market_values = salaries/0.1

# Set up all players
# use binary var cube
# (time, player, and simulation)


# set up goalkeeping separately
# set up DF, MF and Fw using separate dataset



In [47]:
# create model
ftcp = Model(name = "Rarita Football Team Composition")

max_age = 40
starting_budget = 1500000
wage_rev = 0.63
rho = 0.05
ret = 0.02

time = np.arange(n_time)
trials = np.arange(n_trials)

In [48]:
# Decision variables
assigned = ftcp.binary_var_matrix(keys1=players.index.values, keys2=time, name = "assign_%s_%s")

budget = ftcp.continuous_var_matrix(keys1=time, keys2=trials, name = "Budget_%s_%s", lb=0)
team_value = ftcp.continuous_var_matrix(keys1=time, keys2=trials, name = "Team Value_%s_%s")

sunset_value = ftcp.continuous_var_list(keys=trials, name = "Sunset Value_%s")
optim_value = ftcp.continuous_var(name = "Optimisation Value")

In [49]:
# Non decision variables
# Total number of players
N_total = ftcp.integer_var_list(keys = time, name = 'Total number of Players_%s', lb=22, ub=25)

# Number of players in each role
N_gk = ftcp.integer_var_list(keys = time, name = "Number of goalkeepers_%s", lb=2, ub=3)
N_df = ftcp.integer_var_list(keys = time, name = "Number of defenders_%s", lb=8, ub=10)
N_mf = ftcp.integer_var_list(keys = time, name = "Number of midfielders_%s", lb=8, ub=10)
N_fw = ftcp.integer_var_list(keys = time, name = "Number of forwards_%s", lb=3, ub=4)       

In [50]:
# Set up Constraints for number of players
for year in time:
    for trial in trials:
        ###
        # Not used in optim, but will be used as constraint to control costs
        ###
        previous_budget = starting_budget if year == 0 else budget[year-1, trial]
        ftcp.add_constraint(budget[year, trial] == previous_budget * (1 + ret)
                            + (1/wage_rev - 1) * ftcp.sum(assigned[index, year] * salaries[trial, index, year] for index in range(n_players)))
        # Team value = Player values - Wages + Borrowing - Lending
        ftcp.add_constraint(team_value[year, trial] == ftcp.sum(assigned[index, year] * market_values[trial, index, year] for index in range(n_players))/((1 + rho)**year)
                            - ftcp.sum(assigned[index, year] * salaries[trial, index, year] for index in range(n_players)))
    
    ftcp.add_constraint(N_gk[year] == ftcp.sum(assigned[index, year] * player['GK'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_df[year] == ftcp.sum(assigned[index, year] * player['DF'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_mf[year] == ftcp.sum(assigned[index, year] * player['MF'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_fw[year] == ftcp.sum(assigned[index, year] * player['FW'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_gk[year] + N_df[year] + N_mf[year] + N_fw[year] == N_total[year])

    for index, player in players.iterrows():
        ftcp.add_constraint(assigned[index, year] * (player['Age'] + year) <= max_age )

for trial in trials:        
    ftcp.add_constraint(sunset_value[trial] == ftcp.sum(assigned[index, time[-1]] * market_values[trial, index, time[-1]] for index in range(n_players))/((1 + rho)**(n_time)))
    
ftcp.add_constraint(optim_value == (ftcp.sum(team_value[x, y] for x in time for y in trials) + ftcp.sum(sunset_value[x] for x in trials))/n_trials)

docplex.mp.LinearConstraint[](Optimisation Value,EQ,0.333Team Value_0_0+0.333Team Value_0_1+0.333Team Value_0_2+0.333Team Value_1_0+0.333Team Value_1_1+0.333Team Value_1_2+0.333Team Value_2_0+0.333Team Value_2_1+0.333Team Value_2_2+0.333Team Value_3_0+0.333Team Value_3_1+0.333Team Value_3_2+0.333Team Value_4_0+0.333Team Value_4_1+0.333Team Value_4_2+0.333Team Value_5_0+0.333Team Value_5_1+0.333Team Value_5_2+0.333Team Value_6_0+0.333Team Value_6_1+0.333Team Value_6_2+0.333Team Value_7_0+0.333Team Value_7_1+0.333Team Value_7_2+0.333Team Value_8_0+0.333Team Value_8_1+0.333Team Value_8_2+0.333Team Value_9_0+0.333Team Value_9_1+0.333Team Value_9_2+0.333Sunset Value_0+0.333Sunset Value_1+0.333Sunset Value_2)

In [51]:
ftcp.maximize(optim_value)
ftcp.print_information()

Model: Rarita Football Team Composition
 - number of variables: 28334
   - binary=28220, integer=50, continuous=64
 - number of constraints: 28334
   - linear=28334
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP


In [52]:
ftcp.solve(log_output=True, clean_before_solve=True)
ftcp.solve_status

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 28281 rows and 761 columns.
Reduced MIP has 53 rows, 27573 columns, and 43352 nonzeros.
Reduced MIP has 27468 binaries, 102 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.23 sec. (76.55 ticks)
Found incumbent of value 3.6315419e+10 after 0.34 sec. (107.83 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 53 rows, 27573 columns, and 43352 nonzeros.
Reduced MIP has 27468 binaries, 102 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.18 sec. (28.37 ticks)
Probing time = 0.05 sec. (8.16 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.21 sec. (27.24 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound 

<JobSolveStatus.OPTIMAL_SOLUTION: 2>

In [53]:
ftcp.print_solution()

objective: 76773606339.710
  "assign_224_0"=1
  "assign_224_1"=1
  "assign_224_2"=1
  "assign_224_3"=1
  "assign_224_4"=1
  "assign_224_5"=1
  "assign_224_6"=1
  "assign_224_7"=1
  "assign_224_8"=1
  "assign_224_9"=1
  "assign_366_0"=1
  "assign_366_1"=1
  "assign_366_2"=1
  "assign_366_3"=1
  "assign_366_4"=1
  "assign_366_5"=1
  "assign_366_6"=1
  "assign_366_7"=1
  "assign_366_8"=1
  "assign_366_9"=1
  "assign_416_8"=1
  "assign_416_9"=1
  "assign_456_0"=1
  "assign_456_1"=1
  "assign_456_2"=1
  "assign_456_3"=1
  "assign_456_4"=1
  "assign_456_8"=1
  "assign_456_9"=1
  "assign_501_0"=1
  "assign_501_1"=1
  "assign_501_2"=1
  "assign_501_3"=1
  "assign_501_4"=1
  "assign_501_5"=1
  "assign_501_6"=1
  "assign_501_7"=1
  "assign_501_8"=1
  "assign_571_0"=1
  "assign_571_1"=1
  "assign_571_2"=1
  "assign_571_3"=1
  "assign_571_4"=1
  "assign_571_5"=1
  "assign_571_6"=1
  "assign_571_7"=1
  "assign_571_8"=1
  "assign_571_9"=1
  "assign_781_0"=1
  "assign_781_1"=1
  "assign_781_2"=1
  "a

In [54]:
# import docplex.mp.conflict_refiner as cr

# ftcp.solve()
# solve_status = ftcp.get_solve_status()
# if solve_status.name in ['INFEASIBLE_SOLUTION', 'INFEASIBLE_OR_UNBOUNDED_SOLUTION']:
#     cref = cr.ConflictRefiner()
#     print('show some of the constraints that can be removed to arrive at a minimal conflict')
#     cref_result = cref.refine_conflict(ftcp, display=True)  # display flag is to show the conflicts

#     cref_result.display()
#     cref_result.as_output_table()